# Assembling a full workflow

As an example of constructing a full workflow for considering a modeling a chemical process, we consider methane combustion:

$$
\textrm{CH}_4 + 2\textrm{O}_2 \rightarrow \textrm{CO}_2 + 2\textrm{H}_2\textrm{O}
$$

We calculate this reaction energy by constructing equilibrium structures of each molecule, calculating the total energy of each molecule in turn, and taking the energy difference between the reactants and products.

In [11]:
import veloxchem as vlx
from rdkit import Chem
from rdkit.Chem import AllChem
import veloxchem as vlx
import py3Dmol as p3d
import numpy as np
import sys

ostream = vlx.OutputStream(sys.stdout)

## Geometry optimization

### UFF optimization

Construct molecules using SMILES strings and perform an UFF optimization.

In [23]:
def smilestoxyz(smiles, opt = True, return_noH = False):
    mol_bare = Chem.MolFromSmiles(smiles)
    mol_full = Chem.AddHs(mol_bare)
    AllChem.EmbedMolecule(mol_full)
    if opt: AllChem.UFFOptimizeMolecule(mol_full)
    if return_noH:
        return Chem.MolToXYZBlock(mol_full), Chem.RemoveHs(mol_full)
    return Chem.MolToXYZBlock(mol_full)

ch4_uff = smilestoxyz('C')
h2o_uff = smilestoxyz('O')
co2_uff = smilestoxyz('O=C=O')
o2_uff = smilestoxyz('O=O')

### xTB optimization

Set up xTB driver and perform xTB optimization.

In [24]:
def xTX_opt(xyz_uff):
    molecule = vlx.Molecule.from_xyz_string(xyz_uff)
    xtb_drv = vlx.XtbDriver()
    method_settings = {'xtb':'gfn2'}
    xtb_drv.set_method(method_settings['xtb'].lower())
    xtb_drv.compute(molecule, ostream)
    xtb_grad_drv = vlx.XtbGradientDriver(xtb_drv)
    xtb_opt_drv = vlx.OptimizationDriver(xtb_grad_drv)
    xtb_opt = xtb_opt_drv.compute(molecule)
    return xtb_opt

ch4_xtb = xTX_opt(ch4_uff)
h2o_xtb = xTX_opt(h2o_uff)
co2_xtb = xTX_opt(co2_uff)
o2_xtb = xTX_opt(o2_uff)

                                                                                                                          
                                                        XTB Driver                                                        
                                                                                                                          

          ...................................................
          :                      SETUP                      :
          :.................................................:
* Info *   Energy   : -4.1733985067 a.u.                                                                                  
* Info *   Gradient : 1.535665e-02 a.u. (RMS)                                                                             
* Info *              1.716946e-02 a.u. (Max)                                                                             
* Info *   Time     : 0.02 sec                                             

* Info * Computing energy and gradient...                                                                                 

          ...................................................
          :                      SETUP                      :
          :.................................................:
          :  # basis functions                   8          :
          :  # atomic orbitals                   8          :
          :  # shells                            6          :
          :  # electrons                         8          :
          :  max. iterations                   280          :
          :  Hamiltonian                  GFN2-xTB          :
          :  restarted?                       true          :
          :  GBSA solvation                  false          :
          :  PC potential                    false          :
          :  electronic temp.          300.0000000     K    :
          :  accuracy                    1.0000000          :
        

* Info * Saving file: veloxchem_opt_2023-01-17T14.40.20_optim.xyz                                                         
* Info * Saving file: veloxchem_opt_2023-01-17T14.40.20.log                                                               
                                                                                                                          
* Info * Geometry optimization completed.                                                                                 
                                                                                                                          
                                              Molecular Geometry (Angstroms)                                              
                                                                                                                          
                          Atom         Coordinate X          Coordinate Y          Coordinate Z                           
                

                                                                                                                          
                                                Optimization Driver Setup                                                 
                                                                                                                          
                                     Coordinate System       :    TRIC                                                    
                                     Constraints             :    No                                                      
                                     Max. Number of Steps    :    300                                                     
                                     Transition State        :    No                                                      
                                     Hessian                 :    never                                                   
                

* Info * Computing energy and gradient...                                                                                 

          ...................................................
          :                      SETUP                      :
          :.................................................:
          :  # basis functions                   6          :
          :  # atomic orbitals                   6          :
          :  # shells                            4          :
          :  # electrons                         8          :
          :  max. iterations                   280          :
          :  Hamiltonian                  GFN2-xTB          :
          :  restarted?                       true          :
          :  GBSA solvation                  false          :
          :  PC potential                    false          :
          :  electronic temp.          300.0000000     K    :
          :  accuracy                    1.0000000          :
        

...

* Info *   Energy   : -10.3084522891 a.u.                                                                                 
* Info *   Gradient : 4.720172e-06 a.u. (RMS)                                                                             
* Info *              5.780978e-06 a.u. (Max)                                                                             
* Info *   Time     : 0.01 sec                                                                                            
                                                                                                                          
* Info * Computing energy and gradient...                                                                                 

          ...................................................
          :                      SETUP                      :
          :.................................................:
          :  # basis functions                  12          :
          :  

                                     *** Time spent in Optimization Driver: 0.20 sec                                      
                                                                                                                          
                                                                                                                          
                                                        XTB Driver                                                        
                                                                                                                          

          ...................................................
          :                      SETUP                      :
          :.................................................:
          :  # basis functions                   8          :
          :  # atomic orbitals                   8          :
          :  # shells                            4          :
          : 

* Info * Computing energy and gradient...                                                                                 

          ...................................................
          :                      SETUP                      :
          :.................................................:
          :  # basis functions                   8          :
          :  # atomic orbitals                   8          :
          :  # shells                            4          :
          :  # electrons                        12          :
          :  max. iterations                   280          :
          :  Hamiltonian                  GFN2-xTB          :
          :  restarted?                       true          :
          :  GBSA solvation                  false          :
          :  PC potential                    false          :
          :  electronic temp.          300.0000000     K    :
          :  accuracy                    1.0000000          :
        

* Info * Computing energy and gradient...                                                                                 

          ...................................................
          :                      SETUP                      :
          :.................................................:
          :  # basis functions                   8          :
          :  # atomic orbitals                   8          :
          :  # shells                            4          :
          :  # electrons                        12          :
          :  max. iterations                   280          :
          :  Hamiltonian                  GFN2-xTB          :
          :  restarted?                       true          :
          :  GBSA solvation                  false          :
          :  PC potential                    false          :
          :  electronic temp.          300.0000000     K    :
          :  accuracy                    1.0000000          :
        

* Info * Saving file: veloxchem_opt_2023-01-17T14.40.20_optim.xyz                                                         
* Info * Saving file: veloxchem_opt_2023-01-17T14.40.20.log                                                               
                                                                                                                          
* Info * Geometry optimization completed.                                                                                 
                                                                                                                          
                                              Molecular Geometry (Angstroms)                                              
                                                                                                                          
                          Atom         Coordinate X          Coordinate Y          Coordinate Z                           
                

### Visualization

Convert to format readable by py3Dmol and visualize structures.

In [29]:
def toxyz(molecule):
    from veloxchem.veloxchemlib import ChemicalElement
    from veloxchem.veloxchemlib import bohr_in_angstroms

    elem_ids = molecule.elem_ids_to_numpy()
    xs = molecule.x_to_numpy() * bohr_in_angstroms()
    ys = molecule.y_to_numpy() * bohr_in_angstroms()
    zs = molecule.z_to_numpy() * bohr_in_angstroms()

    xyz = "" 
    xyz += "%d\n\n" % molecule.number_of_atoms()
    for elem_id, x, y, z in zip(elem_ids, xs, ys, zs):
        elem = ChemicalElement()
        elem.set_atom_type(elem_id)
        xyz += "%6s %22.12f %22.12f %22.12f\n" % (elem.get_name(),x,y,z)
    return xyz

ch4_xyz = toxyz(ch4_xtb)
h2o_xyz = toxyz(h2o_xtb)
co2_xyz = toxyz(co2_xtb)
o2_xyz = toxyz(o2_xtb)

viewer = p3d.view(viewergrid=(2, 2), width=400, height=300, linked=False)
viewer.addModel(ch4_xyz, 'xyz', viewer=(0, 0))
viewer.addModel(o2_xyz, 'xyz', viewer=(0, 1))
viewer.addModel(co2_xyz, 'xyz', viewer=(1, 0))
viewer.addModel(h2o_xyz, 'xyz', viewer=(1, 1))
viewer.setViewStyle({"style": "outline", "color": "black", "width": 0.1})
viewer.setStyle({"stick": {}})
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Print distance matrices.

In [30]:
print(r'CH$_4$:')
ch4_rdfit = AllChem.MolFromXYZBlock(ch4_xyz)
ch4_dm = AllChem.Get3DDistanceMatrix(ch4_rdfit)
print(np.around(ch4_dm,3))

print(r'O$_2$:')
o2_rdfit = AllChem.MolFromXYZBlock(o2_xyz)
o2_dm = AllChem.Get3DDistanceMatrix(o2_rdfit)
print(np.around(o2_dm,3))

print(r'CO$_2$:')
co2_rdfit = AllChem.MolFromXYZBlock(co2_xyz)
co2_dm = AllChem.Get3DDistanceMatrix(co2_rdfit)
print(np.around(co2_dm,3))

print(r'H$_2$O:')
h2o_rdfit = AllChem.MolFromXYZBlock(h2o_xyz)
h2o_dm = AllChem.Get3DDistanceMatrix(h2o_rdfit)
print(np.around(h2o_dm,3))

CH$_4$:
[[0.    1.082 1.082 1.082 1.082]
 [1.082 0.    1.767 1.767 1.767]
 [1.082 1.767 0.    1.767 1.767]
 [1.082 1.767 1.767 0.    1.767]
 [1.082 1.767 1.767 1.767 0.   ]]
O$_2$:
[[0.   1.21]
 [1.21 0.  ]]
CO$_2$:
[[0.    1.144 2.287]
 [1.144 0.    1.144]
 [2.287 1.144 0.   ]]
H$_2$O:
[[0.    0.959 0.959]
 [0.959 0.    1.544]
 [0.959 1.544 0.   ]]


## Energy calculation

Calculate the total energy of each molecule using B3LYP and a 6-311G* basis set.

```{note}
Keep in mind that O$_2$ has a triplet ground state, which is here set by changing the multiplicity and using unrestricted SCF.
```

In [32]:
def scf_calculation(xyz, basis_set, xc, multiplicity = None):
    molecule = vlx.Molecule.from_xyz_string(xyz)
    basis = vlx.MolecularBasis.read(molecule, basis_set)
    if multiplicity:
        molecule.set_multiplicity(multiplicity)
        scf_gs = vlx.ScfUnrestrictedDriver()
    else:
        scf_gs = vlx.ScfRestrictedDriver()
    scf_gs.xcfun = xc
    scf_results = scf_gs.compute(molecule, basis)
    return scf_results, scf_gs

xc = 'b3lyp'
basis_set = '6-311G*'

ch4_results, ch4_scf = scf_calculation(ch4_xyz, basis_set, xc)
o2_results, o2_scf = scf_calculation(o2_xyz, basis_set, xc, multiplicity = 3)
co2_results, co2_scf = scf_calculation(co2_xyz, basis_set, xc)
h2o_results, h2o_scf = scf_calculation(h2o_xyz, basis_set, xc)

* Info * Reading basis set from file: /home/thomas/Notebook/anaconda/envs/echem_test/lib/python3.9/site-packages/veloxchem/basis/6-311G_D_
                                                                                                                          
                                              Molecular Basis (Atomic Basis)                                              
                                                                                                                          
                               Basis: 6-311G*                                                                             
                                                                                                                          
                               Atom Contracted GTOs           Primitive GTOs                                              
                                                                                                                          


               Spin-Restricted Kohn-Sham:                                                                                 
               --------------------------                                                                                 
               Total Energy                       :      -40.5277651170 a.u.                                              
               Electronic Energy                  :      -54.0597643535 a.u.                                              
               Nuclear Repulsion Energy           :       13.5319992365 a.u.                                              
               ------------------------------------                                                                       
               Gradient Norm                      :        0.0000000970 a.u.                                              
                                                                                                                          
                

* Info * Reading basis set from file: /home/thomas/Notebook/anaconda/envs/echem_test/lib/python3.9/site-packages/veloxchem/basis/6-311G_D_
                                                                                                                          
                                              Molecular Basis (Atomic Basis)                                              
                                                                                                                          
                               Basis: 6-311G*                                                                             
                                                                                                                          
                               Atom Contracted GTOs           Primitive GTOs                                              
                                                                                                                          


               Spin-Unrestricted Kohn-Sham:                                                                               
               ----------------------------                                                                               
               Total Energy                       :     -150.3647703314 a.u.                                              
               Electronic Energy                  :     -178.3586648551 a.u.                                              
               Nuclear Repulsion Energy           :       27.9938945237 a.u.                                              
               ------------------------------------                                                                       
               Gradient Norm                      :        0.0000004182 a.u.                                              
                                                                                                                          
                

* Info * Reading basis set from file: /home/thomas/Notebook/anaconda/envs/echem_test/lib/python3.9/site-packages/veloxchem/basis/6-311G_D_
                                                                                                                          
                                              Molecular Basis (Atomic Basis)                                              
                                                                                                                          
                               Basis: 6-311G*                                                                             
                                                                                                                          
                               Atom Contracted GTOs           Primitive GTOs                                              
                                                                                                                          


                  8      -188.639946865483    0.0000000000      0.00000030      0.00000004      0.00000066                
                                                                                                                          
               *** SCF converged in 8 iterations. Time: 4.08 sec.                                                         
                                                                                                                          
               Spin-Restricted Kohn-Sham:                                                                                 
               --------------------------                                                                                 
               Total Energy                       :     -188.6399468655 a.u.                                              
               Electronic Energy                  :     -247.8666377327 a.u.                                              
               N

* Info * Reading basis set from file: /home/thomas/Notebook/anaconda/envs/echem_test/lib/python3.9/site-packages/veloxchem/basis/6-311G_D_
                                                                                                                          
                                              Molecular Basis (Atomic Basis)                                              
                                                                                                                          
                               Basis: 6-311G*                                                                             
                                                                                                                          
                               Atom Contracted GTOs           Primitive GTOs                                              
                                                                                                                          


                                                                                                                          
               *** SCF converged in 7 iterations. Time: 1.67 sec.                                                         
                                                                                                                          
               Spin-Restricted Kohn-Sham:                                                                                 
               --------------------------                                                                                 
               Total Energy                       :      -76.4338897463 a.u.                                              
               Electronic Energy                  :      -85.6033485163 a.u.                                              
               Nuclear Repulsion Energy           :        9.1694587700 a.u.                                              
               -

## Reaction energy

The reaction energy is now taken as the energy difference between the reactants and products:

In [34]:
e_reactants = ch4_scf.get_scf_energy() + 2*o2_scf.get_scf_energy()
e_products = co2_scf.get_scf_energy() + 2*h2o_scf.get_scf_energy()

print(f'The reaction energy is {627.5*(e_products - e_reactants):.1f} kcal/mol')

The reaction energy is -157.1 kcal/mol


This can be compared to the results presented in [this](https://doi.org/10.1021/acs.jchemed.1c01243) paper, which discusses the question of computational cost versus accuracy with methane combustion as an example. They obtain combustion energies ranging from approximately -157.3 kcal/mol when using Hartree--Fock and a small basis set, which is compared to a reference estimates of -193.2 kcal/mol, obtained using high-level theory.


In order to investigate the combustion energy of any other alkane, we merely need to change the corresponding SMILES string and calculating energy differences according to the general reaction equation:

$$
2\textrm{C}_n\textrm{H}_{2n+2} + (3n+1)\textrm{O}_2 \rightarrow 2n\textrm{CO}_2 + 2(n+1)\textrm{H}_2\textrm{O}
$$

With this, we obtain reaction energies for the first five hydrocarbons, considering both the total energy and the energy per carbon atom:


| Hydrocarbon  | kcal/mol | kcal/mol per carbon |
| -----------  | -------- | -------------- |
|    methane   |  -157.1  |    -157.1      |
|     ethane   |  -286.8  |    -143.4      |
|    propane   |  -415.4  |    -138.8      |
|    butane    |  -544.9  |    -136.2      |
|    heptane   |  -673.7  |    -134.7      |

